# urb_percep

In [1]:
import pandas as pd
from functools import reduce #necessary to create the last dataframe
df = pd.read_csv(r"C:\Users\hp pc\Desktop\Tesi\Smart cities\Databases\1. Technology\ICT Structures\urb_percep.csv", encoding = "ISO-8859-1",index_col=0)
del df["UNIT"] #Delete the unit of measurement
# Rename cities in English
df = df.replace(["Athina"],"Athens")
df = df.replace(['Bruxelles / Brussel'],'Bruxelles')
df = df.replace(['København'],'Copenhagen')
df = df.replace(['Helsinki / Helsingfors'],'Helsinki')
df = df.replace(['Madrid'],'Madrid')
df = df.replace(['Paris'],'Paris')
df = df.replace(['Praha'],'Prague')
df = df.replace(['Roma'],'Rome')
df = df.replace(['Stockholm'],'Stockholm')
df = df.replace(['Warszawa'],'Warsaw')
df = df.replace(['Wien'],'Vienna')
df = df.replace(['Zagreb'],'Zagreb')

#these are the cities that we analyse in our study
cities=["Athens","Bruxelles","Copenhagen", "Helsinki", "Madrid", "Paris", "Prague", "Rome", 
        "Stockholm", "Warsaw","Vienna", "Zagreb"]

df.sort_values(by=['CITIES'], inplace=True, ascending=True) # sort by cities
df["TIME"]=df.index # Create a column with the date, which is set as index by default
df.reset_index(drop=True, inplace=True)  # reset index
indicators = list(dict.fromkeys(df["INDIC_UR"])) # make list containing names of the variables (no duplicates)

dfList=[] #create an empty list that will filled with dataframes
for nome_indicatore in indicators:
    df1=df[df.INDIC_UR == nome_indicatore] 
    for city in cities: 
        if df1[df1['CITIES'].str.contains(city)].empty: #if one city in our sample is not in the df:
            unav_cities.append(((city +" has no data for " + nome_indicatore))) # put the name of the city in the list of cities with data NA
    #create a subdataframe for each indicator
        else:
            sub_df=df1[df1.CITIES == city] #take the subdataframe with the name of each city 
            sub_df=sub_df[sub_df.TIME==max(sub_df["TIME"])] # get the row where date is the most recent
            dfList.append(sub_df)
df=(pd.concat(dfList,ignore_index=False)) # This df containes all variables: it needs to be splitted...

final_dfs=[]     #...split the previous df and creating the complete one
for indicator in indicators:
    df1=df[df.INDIC_UR == indicator] 
    df1 = df1.rename(columns={"Value": indicator}) #rename "Value" with the name of the variable 
    del df1["INDIC_UR"] # delete the column filled with the name of the variable. 
    df1.reset_index(drop=True, inplace=True) 
    df1.sort_values("CITIES", axis=0, ascending=True, inplace=True)
    final_dfs.append(df1) # this is the list of df that need to be merged
df= reduce(lambda x, y: pd.merge(x, y, on = 'CITIES'), final_dfs) #merge dfs on CITIES one next to the other
del df["TIME_y"] # (before deleting check if columns are equals)
del df["TIME_x"]

df

,CITIES,Internet access at home in the city: rather satisfied,Internet access at home in the city: very satisfied,Public Internet access such as internet cafes or libraries in the city: rather satisfied,Public Internet access such as internet cafes or libraries in the city: very satisfied
0,Athens,15.3,26.6,17.1,14.9
1,Bruxelles,24.9,34.4,21.2,19.5
2,Copenhagen,34.2,51.0,41.1,21.2
3,Helsinki,24.0,61.2,36.0,18.6
4,Madrid,37.0,14.4,31.3,7.6
5,Paris,34.0,36.7,24.8,12.8
6,Prague,26.6,46.5,24.3,14.7
7,Rome,29.2,27.8,25.2,10.2
8,Stockholm,35.6,37.6,26.2,12.7
9,Vienna,19.2,49.3,24.5,19.6


In [2]:
# RENAME COLUMNS USING EASIER NAMES

df = df.rename(columns={"Internet access at home in the city: rather satisfied": "Int_home_rather"})
df = df.rename(columns={"Internet access at home in the city: very satisfied": "Int_home_very"})

df = df.rename(columns={"Public Internet access such as internet cafes or libraries in the city: rather satisfied": "Pub_int_rather"})
df = df.rename(columns={"Public Internet access such as internet cafes or libraries in the city: very satisfied": "Pub_int_very"})

# AND MERGE (ADD) "RATHER + VERY SATISFIED" 
df["Int_home"]=df["Int_home_rather"] + df["Int_home_very"] 
df["Pub_int"]=df["Pub_int_rather"] + df["Pub_int_very"] 

# AND DELETE USELESS COLUMNS
del df["Int_home_rather"]
del df["Int_home_very"]
del df["Pub_int_rather"]
del df["Pub_int_very"]

df

,CITIES,Int_home,Pub_int
0,Athens,41.9,32.0
1,Bruxelles,59.3,40.7
2,Copenhagen,85.2,62.3
3,Helsinki,85.2,54.6
4,Madrid,51.4,38.9
5,Paris,70.7,37.6
6,Prague,73.1,39.0
7,Rome,57.0,35.4
8,Stockholm,73.2,38.9
9,Vienna,68.5,44.1


In [3]:
# # Apply min-max normalization formula to each value in df 
df["Int_home"]=[round(((s-min(df.Int_home))/(max(df.Int_home)-min(df.Int_home)))*10,1) for s in df["Int_home"]]
df["Pub_int"]=[round(((s-min(df.Pub_int))/(max(df.Pub_int)-min(df.Pub_int)))*10,1) for s in df["Pub_int"]]
urb_percep=df
urb_percep

,CITIES,Int_home,Pub_int
0,Athens,0.0,0.0
1,Bruxelles,4.0,2.9
2,Copenhagen,10.0,10.0
3,Helsinki,10.0,7.5
4,Madrid,2.2,2.3
5,Paris,6.7,1.8
6,Prague,7.2,2.3
7,Rome,3.5,1.1
8,Stockholm,7.2,2.3
9,Vienna,6.1,4.0


# % of Households with internet access 

In [4]:
import pandas as pd
from functools import reduce #necessary to create the last dataframe
df = pd.read_csv(r"C:\Users\hp pc\Desktop\Tesi\Smart cities\Databases\1. Technology\ICT Structures\isoc_r_iacc_h.csv", encoding = "ISO-8859-1",index_col=0)
del df["UNIT"] #Delete the unit of measurement
# Rename cities in English. This Dataframe contains regional data
df = df.replace(["Attiki"],"Athens")
df = df.replace(['Région de Bruxelles-Capitale / Brussels Hoofdstedelijk Gewest'],'Bruxelles')
df = df.replace(['Hovedstaden'],'Copenhagen')
df = df.replace(['Helsinki-Uusimaa'],'Helsinki')
df = df.replace(['Comunidad de Madrid'],'Madrid')
df = df.replace(['Île de France'],'Paris')
df = df.replace(['Praha'],'Prague')
df = df.replace(['Lazio'],'Rome')
df = df.replace(['Stockholm'],'Stockholm')
df = df.replace(['Makroregion Województwo Mazowieckie'],'Warsaw')
df = df.replace(['Wien'],'Vienna')
df = df.replace(['Hrvatska'],'Zagreb')

#these are the cities that we analyse in our study
cities=["Athens","Bruxelles","Copenhagen", "Helsinki", "Madrid", "Paris", "Prague", "Rome", "Stockholm",
        "Warsaw","Vienna", "Zagreb"]
df = df.rename(columns={"GEO": "CITIES"}) # RENAME COLUMNS
df = df.rename(columns={"Value": "% of Households with internet access"}) 
df.sort_values("CITIES", axis=0, ascending=True, inplace=True)
df

,CITIES,% of Households with internet access
TIME,,
2019,Athens,84
2019,Bruxelles,89
2019,Copenhagen,95
2019,Helsinki,97
2019,Madrid,95
2019,Paris,93
2019,Prague,93
2019,Rome,88
2019,Stockholm,95


In [5]:
# RENAME COLUMNS USING EASIER NAMES 
df = df.rename(columns={"% of Households with internet access": "House_int_acc"})
# Apply min-max normalization formula to each value in df
df = df.rename(columns={ "Individuals who accessed the internet away from home or work": "Int_Not_home_work"}) 
df["House_int_acc"]=[round(((s-min(df.House_int_acc))/(max(df.House_int_acc)-min(df.House_int_acc)))*10,1) for s in df["House_int_acc"]]
df.reset_index(drop=True, inplace=True) 
isoc_r_iacc_h=df
isoc_r_iacc_h

,CITIES,House_int_acc
0,Athens,1.9
1,Bruxelles,5.0
2,Copenhagen,8.8
3,Helsinki,10.0
4,Madrid,8.8
5,Paris,7.5
6,Prague,7.5
7,Rome,4.4
8,Stockholm,8.8
9,Vienna,6.9


# of households with computers (manca nel caso cancellare)

# ICT INDEX

In [6]:
# MERGE DATAFRAME CREATE AN INDEX FOR ICT
result = pd.merge(urb_percep, isoc_r_iacc_h, on=['CITIES'])
result

,CITIES,Int_home,Pub_int,House_int_acc
0,Athens,0.0,0.0,1.9
1,Bruxelles,4.0,2.9,5.0
2,Copenhagen,10.0,10.0,8.8
3,Helsinki,10.0,7.5,10.0
4,Madrid,2.2,2.3,8.8
5,Paris,6.7,1.8,7.5
6,Prague,7.2,2.3,7.5
7,Rome,3.5,1.1,4.4
8,Stockholm,7.2,2.3,8.8
9,Vienna,6.1,4.0,6.9


In [8]:
result['ICT_Structures_Index'] = round((result.mean(axis=1)),2)
ICT_Structures_Index=result[["CITIES", "ICT_Structures_Index"]]
ICT_Structures_Index.sort_values(by=['ICT_Structures_Index'], inplace=True, ascending=False)
ICT_Structures_Index

C:\Users\hp pc\Documents\Python-Anaconda\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,CITIES,ICT_Structures_Index
2,Copenhagen,9.60
3,Helsinki,9.17
8,Stockholm,6.10
6,Prague,5.67
9,Vienna,5.67
5,Paris,5.33
4,Madrid,4.43
1,Bruxelles,3.97
10,Warsaw,3.70
7,Rome,3.00


In [9]:
%store ICT_Structures_Index

Stored 'ICT_Structures_Index' (DataFrame)
